In [1]:
import os
print(os.environ['CONDA_DEFAULT_ENV'])
#.\venv_mediapipe_protobuf\Scripts\activate    to activate virtual env 

opencv-env


In [2]:
#pip install numpy~=1.26.4 scikit-learn~=1.5.0 tensorflow~=2.16.1 opencv-python~=4.10.0.82 mediapipe~=0.10.14 pandas~=2.2.2 pytube~=15.0.0

In [3]:
#!pip install pytube


In [1]:
#  from google.protobuf import symbol_database
#  from google.protobuf import message_factory

#  class PatchedSymbolDatabase(symbol_database.SymbolDatabase):
#      def GetPrototype(self, descriptor):
# #         return message_factory.GetMessageClass(descriptor)

# # # Replace the default SymbolDatabase instance with the patched one
# # symbol_database._DEFAULT = PatchedSymbolDatabase(pool=symbol_database._DEFAULT.pool)


In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import mediapipe as mp
import time
from pytube import YouTube


In [2]:
# Extracting feature keypoints using MediaPipe holistic model
mp_holistic = mp.solutions.holistic  # Holistic model
mp_drawing_styles = mp.solutions.drawing_styles
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities


In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert image from BGR to RGB
    image.flags.writeable = False  # Set the 'writable' flag to False to improve performance
    results = model.process(image)  # Make predictions
    image.flags.writeable = True  # Set the 'writable' flag back to True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert image back to BGR
    return image, results



In [4]:
def get_youtube_stream_url(video_url):
    yt = YouTube(video_url)
    stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
    return stream.url


In [5]:
def draw_styled_landmarks(image, results):
    # Define drawing specs for each landmark type
    face_spec = mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=1, circle_radius=1)  # red, thickness 1, circle radius 1
    pose_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)  # green, thickness 2, circle radius 2
    hand_spec = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)  # blue, thickness 2, circle radius 2

    # Draw styled landmarks
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, face_spec)
         # mp.solutions.drawing_utils.draw_landmarks(
    #     image, results.face_landmarks, mp.solutions.holistic.FACEMESH_CONTOURS,
    #      face_spec)
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, pose_spec)
        # mp.solutions.drawing_utils.draw_landmarks(
    #     image, results.pose_landmarks, mp.solutions.holistic.POSE_CONNECTIONS,
    #      pose_spec)
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, hand_spec)
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, hand_spec)

  
    # # Draw face connections
   
    # # Draw pose connections
    

   

In [6]:
def capture_frames(folder_name, frame, count):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    frame_path = os.path.join(folder_name, f"frame_{count}.JPEG")
    cv2.imwrite(frame_path, frame)
    print(f"Captured {frame_path}")


In [7]:
def primary_data_capture():
    holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5, enable_segmentation=True)
    cap = cv2.VideoCapture(0)
    window_name = "Holistic Model Detection"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

    folder_name = input("Enter the main folder name (e.g., 'data_collection'): ")
    class_subfolder_name = input("Enter the class subfolder name (e.g., 'Alphabets'): ")
    frame_subfolder_name = input("Enter the frame subfolder name (e.g., 'A'): ")

    full_path = os.path.join(folder_name, class_subfolder_name, frame_subfolder_name)

    if not os.path.exists(full_path):
        os.makedirs(full_path)

    existing_files = [f for f in os.listdir(full_path) if os.path.isfile(os.path.join(full_path, f))]
    if existing_files:
        existing_files.sort(key=lambda f: int(f.split('_')[1].split('.')[0]))
        last_frame_num = int(existing_files[-1].split('_')[1].split('.')[0])
    else:
        last_frame_num = 0

   
    frame_count = last_frame_num + 1
    paused = False
    rewind = False
    forward = False

    while cap.isOpened():
        if not paused:
            if rewind:
                current_frame = max(0, cap.get(cv2.CAP_PROP_POS_FRAMES) - 20)
                cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
                rewind = False  # Reset rewind after applying it
            elif forward:
                current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES) + 60
                cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
                forward = False  # Reset forward after applying it
            
            ret, frame = cap.read()
            if not ret:
                break

            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)

            cv2.imshow("Holistic Model Detection", image)

            if results.segmentation_mask is not None:
                segmentation_mask_frame = cv2.cvtColor(results.segmentation_mask, cv2.COLOR_GRAY2BGR)
                cv2.imshow("Segmentation Mask", segmentation_mask_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            capture_frames(full_path, frame, frame_count)
            frame_count += 1
        elif key == ord(' '):  # Space key to pause/play
            paused = not paused
        elif key == ord('r'):
            rewind = True
        elif key == ord('f'):
            forward = True
        elif key == ord('q') or key == 27:  # 'q' or Escape key to quit
            break

    cap.release()
    cv2.destroyAllWindows()


In [8]:
def secondary_data_capture(video_url):
    stream_url = get_youtube_stream_url(video_url)
    cap = cv2.VideoCapture(stream_url)
    holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5, enable_segmentation=True)

    folder_name = input("Enter the main folder name (e.g., 'data_collection'): ")
    class_subfolder_name = input("Enter the class subfolder name (e.g., 'Alphabets'): ")
    frame_subfolder_name = input("Enter the frame subfolder name (e.g., 'A'): ")

    full_path = os.path.join(folder_name, class_subfolder_name, frame_subfolder_name)

    if not os.path.exists(full_path):
        os.makedirs(full_path)

    existing_files = [f for f in os.listdir(full_path) if os.path.isfile(os.path.join(full_path, f))]
    if existing_files:
        existing_files.sort(key=lambda f: int(f.split('_')[1].split('.')[0]))
        last_frame_num = int(existing_files[-1].split('_')[1].split('.')[0])
    else:
        last_frame_num = 0

    frame_count = last_frame_num + 1
    paused = False
    rewind = False
    forward = False

    while cap.isOpened():
        if not paused:
            if rewind:
                current_frame = max(0, cap.get(cv2.CAP_PROP_POS_FRAMES) - 20)
                cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
                rewind = False  # Reset rewind after applying it
            elif forward:
                current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES) + 120
                cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
                forward = False  # Reset forward after applying it
            
            ret, frame = cap.read()
            if not ret:
                break

            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)

            cv2.imshow("Holistic Model Detection", image)

            if results.segmentation_mask is not None:
                segmentation_mask_frame = cv2.cvtColor(results.segmentation_mask, cv2.COLOR_GRAY2BGR)
                cv2.imshow("Segmentation Mask", segmentation_mask_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            capture_frames(full_path, frame, frame_count)
            frame_count += 1
        elif key == ord(' '):  # Space key to pause/play
            paused = not paused
        elif key == ord('r'):
            rewind = True
        elif key == ord('f'):
            forward = True
        elif key == ord('q') or key == 27:  # 'q' or Escape key to quit
            break

    cap.release()
    cv2.destroyAllWindows()


In [15]:
def main():
    print("Choose data capture method:")
    print("Press 'p' for Primary Data Capture (camera)")
    print("Press 's' for Secondary Data Capture (YouTube video)")

    cv2.namedWindow("Choose Data Capture Method", cv2.WINDOW_NORMAL)
    cv2.imshow("Choose Data Capture Method", np.zeros((100, 200, 3), dtype=np.uint8))

    while True:
        key = cv2.waitKey(1) & 0xFF
        if key == ord('p'):
            primary_data_capture()
            break
        elif key == ord('s'):
            video_url = input("Enter YouTube video URL: ")
            secondary_data_capture(video_url)
            break

    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

Choose data capture method:
Press 'p' for Primary Data Capture (camera)
Press 's' for Secondary Data Capture (YouTube video)


Enter YouTube video URL:  https://youtu.be/tKY-1NkAS0o?si=StpxYP_LDrws2AQ-
Enter the main folder name (e.g., 'data_collection'):  data collection
Enter the class subfolder name (e.g., 'Alphabets'):  alphabets
Enter the frame subfolder name (e.g., 'A'):  G


Captured data collection\alphabets\G\frame_16.JPEG
Captured data collection\alphabets\G\frame_17.JPEG
Captured data collection\alphabets\G\frame_18.JPEG
Captured data collection\alphabets\G\frame_19.JPEG
Captured data collection\alphabets\G\frame_20.JPEG
Captured data collection\alphabets\G\frame_21.JPEG
Captured data collection\alphabets\G\frame_22.JPEG
Captured data collection\alphabets\G\frame_23.JPEG
Captured data collection\alphabets\G\frame_24.JPEG
Captured data collection\alphabets\G\frame_25.JPEG
Captured data collection\alphabets\G\frame_26.JPEG
Captured data collection\alphabets\G\frame_27.JPEG
Captured data collection\alphabets\G\frame_28.JPEG
Captured data collection\alphabets\G\frame_29.JPEG
Captured data collection\alphabets\G\frame_30.JPEG
